In [1]:
# -*- coding: utf-8 -*-

#@author: alison

import re
import string
import pickle
import keras
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, GRU, CuDNNGRU
from keras.layers import Input, Activation
from keras.optimizers import Adam, SGD, RMSprop
from keras import optimizers
from keras import regularizers

Using TensorFlow backend.


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
file_list = drive.ListFile({'q': "'1Hx5OP1Yrlh37yYzSMtsv6Ui_fuzOuG04' in parents and trashed=false"}).GetList()
for file1 in file_list:
    print('title: %s, id: %s' % (file1['title'], file1['id']))

title: WordEmbedding-GRU_en_a.ipynb, id: 1fcw8EGX2Hdz-tTbz0NDEhZlOHiPvKNsY
title: bi_lstm_en_a.ipynb, id: 1O-dGg5mtIxPsI4JY16dcp4ytezzN5NQZ
title: lstm_en_a.ipynb, id: 1w3Rq-g3l6u5SsBoXoNcKeciE0YUMLJcP
title: gru_en_a.ipynb, id: 1qHxV1RB5vy_M889397xa1Ep4kEx_9RIa
title: CNN_es_a.ipynb, id: 1UDY2L6Ap8rqV0HtWw9tnLICN4MUD3Nno
title: WordEmbedding-biLSTM_es_a.ipynb, id: 14eSqh0xSpcRJXYwU8Qkv9eJaSW_dt3zb
title: WordEmbedding-LSTM_es_a.ipynb, id: 16FhVQWB5WvbRXDGkDgsLJWOTqyKwaDwb
title: test_es.tsv, id: 1sg7mAwSd4KQD9YXmV_ktcZ_gfygUeJaq
title: test_en.tsv, id: 1ofz9ODrpeQMmqwhc8LQK_icexbNdNF7u
title: WordEmbedding-GRU_es_a.ipynb, id: 1ahrxlOEINVnCp05yCjmLx8b4v-5SSGL6
title: bi_lstm_es_a.ipynb, id: 12E3oKtx2N9XPUqK2XXqxMHmxK2EpYdAA
title: lstm_es_a.ipynb, id: 1yZRPqV6WKQ8OeNwcfpQwj0jqiIMehAHK
title: gru_es_a.ipynb, id: 11DtYTZbYyKHgTFIMXhy8Y4I7c3_0z_W0
title: CNN_en_a.ipynb, id: 19oqoK5aiZrBvFOPN1v89xNcXLevjJY0s
title: WordEmbedding-biLSTM_en_a.ipynb, id: 1JOBvoH52NYgevXOGS-K5Cw4ggR4VYWZP
titl

In [0]:
train_downloaded = drive.CreateFile({'id': '1TIjlRkVNIvM8NL3P-4UAmMY6moF3mff0'})
train_downloaded.GetContentFile('train_en.tsv')
dev_downloaded = drive.CreateFile({'id': '1QqOc_95fjvjbw7-uYT37veooqMzfYn-p'})
dev_downloaded.GetContentFile('dev_en.tsv')
test_downloaded = drive.CreateFile({'id': '1ofz9ODrpeQMmqwhc8LQK_icexbNdNF7u'})
test_downloaded.GetContentFile('test_en.tsv')

In [0]:
train = pd.read_csv('train_en.tsv', delimiter='\t',encoding='utf-8')
dev = pd.read_csv('dev_en.tsv', delimiter='\t',encoding='utf-8')
#test = pd.read_csv('test_en.tsv', delimiter='\t',encoding='utf-8')

In [0]:
# Etapa de pré-processamento

def clean_tweets(tweet):
    tweet = re.sub('@(\\w{1,15})\b', '', tweet)
    tweet = tweet.replace("via ", "")
    tweet = tweet.replace("RT ", "")
    tweet = tweet.lower()
    return tweet
    
def clean_url(tweet):
    tweet = re.sub('http\\S+', '', tweet, flags=re.MULTILINE)   
    return tweet
    
def remove_stop_words(tweet):
    stops = set(stopwords.words("english"))
    stops.update(['.',',','"',"'",'?',':',';','(',')','[',']','{','}'])
    toks = [tok for tok in tweet if not tok in stops and len(tok) >= 3]
    return toks
    
def stemming_tweets(tweet):
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tweet]
    return stemmed_words

def remove_number(tweet):
    newTweet = re.sub('\\d+', '', tweet)
    return newTweet

def remove_hashtags(tweet):
    result = ''

    for word in tweet.split():
        if word.startswith('#') or word.startswith('@'):
            result += word[1:]
            result += ' '
        else:
            result += word
            result += ' '

    return result

In [0]:
def preprocessing(tweet, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True):

    if ctweets:
        tweet = clean_tweets(tweet)

    if url:
        tweet = clean_url(tweet)

    if hashtag:
        tweet = remove_hashtags(tweet)
    
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)

    if number:
        tweet = remove_number(tweet)
    
    tokens = [w.lower() for w in twtk.tokenize(tweet) if w != "" and w is not None]

    if swords:
        tokens = remove_stop_words(tokens)

    if stemming:
        tokens = stemming_tweets(tokens)

    text = " ".join(tokens)

    return text

In [0]:
train_text  = train['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
y_train  = train['HS']
id_train = train['id']

test_text  = dev['text'].map(lambda x: preprocessing(x, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True))
y_test  = dev['HS']
id_test = dev['id']

data = np.concatenate((train_text, test_text), axis=0)
classes = np.concatenate((y_train, y_test), axis=0)

In [0]:
word_embedding_dim = 300
max_features = 25000
maxlen = 100
batch_size = 32
epochs = 5

# Treina um tokenizaddor nos dados de treino
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data)

# Tokeniza os dados
X = tokenizer.texts_to_sequences(data)
Y = tokenizer.texts_to_sequences(test_text)

tweets = sequence.pad_sequences(X, maxlen=maxlen)
x_test = sequence.pad_sequences(Y, maxlen=maxlen)

word_index = tokenizer.word_index

In [0]:
x_train, x_val, y_train, y_val = train_test_split(tweets, classes, test_size=0.25, random_state=None)

In [0]:
num_words = min(max_features, len(word_index) + 1)

In [0]:
from tqdm import tqdm

In [14]:
file_list = drive.ListFile({'q': "'1sBKK1i4JXIluelnAPqjOU4xVIIPA3Vmy' in parents and trashed=false"}).GetList()
for file1 in file_list:
    print('title: %s, id: %s' % (file1['title'], file1['id']))

title: GoogleNews-word2vec-300_en.bin, id: 1bERzzLRCcPdavQYVJCwLvt8uWmiRdg_o
title: glove-300d_es.vec, id: 167op9mvIKVfH1XGm3dkZ7afHReiqvmIN
title: glove.6B.300d_en.txt, id: 1FRfAM3GouOxelBo_gwj5n4YNWDMP6CkL
title: wiki_fasttext_es.vec, id: 1S1ObfL0R15IzRK_T1RrSxZMn0TJDECsN
title: wiki_fasttext_en.vec, id: 18zPA_9tWmlNZChdfMKHTp0Ka1SQsnkj9
title: TaskA, id: 1Hx5OP1Yrlh37yYzSMtsv6Ui_fuzOuG04
title: TaskB, id: 1JsHac7_mxx_M8BapYSzcDKnzl4M9e1Yb


In [0]:
we    = {'id': '1FRfAM3GouOxelBo_gwj5n4YNWDMP6CkL', 'file': 'glove.6B.300d_en.txt'}
#we = {'id': '18zPA_9tWmlNZChdfMKHTp0Ka1SQsnkj9', 'file': 'wiki_fasttext_en.vec'}

In [0]:
word_vector = drive.CreateFile({'id': we['id']})
word_vector.GetContentFile(we['file'])

In [0]:
def word_embeddings(word_index, num_words, word_embedding_dim):
    embeddings_index = {}
    
    f = open(we['file'], 'r', encoding='utf-8')
    
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    
    f.close()

    embedding_matrix = np.zeros((num_words, word_embedding_dim))
    
    for word, i in word_index.items():
        if i >= max_features:
            continue
        
        embedding_vector = embeddings_index.get(word)
        
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [27]:
weights_embedding_matrix = word_embeddings(word_index, num_words, word_embedding_dim)

400000it [00:33, 12044.87it/s]


In [28]:
# Fase de classificação de sentimentos

tweet_input = Input(shape=(maxlen,), dtype='int32')

embedding = Embedding(num_words, word_embedding_dim, input_length=maxlen, weights=[weights_embedding_matrix], trainable=True)(tweet_input)

gru1 = CuDNNGRU(256, kernel_initializer='normal', return_sequences=True)(embedding)

gru2 = CuDNNGRU(256)(gru1)

dens = Dense(256, activation='relu')(gru2)

output = Dense(1, activation='sigmoid')(dens)

model = Model(inputs=tweet_input, outputs=output)

#opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.825, epsilon=1e-08)
#opt = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
opt = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.01)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_data=(x_val, y_val))

y_pred = (model.predict(x_test, batch_size=batch_size) > .5).astype(int)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 300)          4993200   
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (None, 100, 256)          428544    
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (None, 256)               394752    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 5,882,545
Trainable params: 5,882,545
Non-trainable params: 0
_________________________________________________________________


In [29]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score

print("F1.........: %f" %(f1_score(y_test, y_pred, average="macro")))
print("Precision..: %f" %(precision_score(y_test, y_pred, average="macro")))
print("Recall.....: %f" %(recall_score(y_test, y_pred, average="macro")))
print("Accuracy...: %f" %(accuracy_score(y_test, y_pred)))

F1.........: 0.832988
Precision..: 0.834012
Recall.....: 0.832107
Accuracy...: 0.837000


In [30]:
output = []
for array in y_pred:
    output.append(array[0])
print(len(output), len(id_test))

1000 1000


In [0]:
from google.colab import files

with open("en_a.tsv", "w") as file:
    for i in range(len(y_pred)):
        file.write(str(id_test[i]))
        file.write('\t')
        file.write(str(output[i]))
        file.write('\n')

files.download('en_a.tsv')